In [51]:
import json
import os
from tqdm import tqdm
from PIL import Image
from PIL import ImageDraw
import json
from multiprocess import Pool
from process_single_file import process_single_file

In [52]:
data_split = ['train','val']

path = '/Users/rajkrishnanv/study/CSC2516/CSC2516_Final_Project/data_subset/'

In [53]:
fl = open('label2id.json')
label2id = json.load(fl)

fl.close()

In [54]:
fl = open('id2label.json')
id2label = json.load(fl)
fl.close()

In [59]:
def process_single_file(args):
    print("inside process_single_file")
    each_file,each_folder,combined_folder_path,new_path = args
    
    print(combined_folder_path + each_folder + '/' +each_file)
    fl = open(combined_folder_path + each_folder + '/' +each_file, 'a+')
    json_obj = json.load(fl)
    fl.close()
    print("f1: ", f1)
    
    json_data = json_obj
    img_height = json_data["imgHeight"]
    img_width = json_data["imgWidth"]

    # Create a blank image
    image = Image.new(mode="L",size= (img_width, img_height))
    draw = ImageDraw.Draw(image)

    # Draw polygons on the image
    for obj in json_data["objects"]:
        if obj["draw"]:
            label = obj["label"]
            correct_id = label2id[label]
            polygon = [(point[0], point[1]) for point in obj["polygon"]]
            draw.polygon(polygon,fill=correct_id)

    image.save(new_path+'/'+each_folder+'/'+each_file.replace("json","png"))
    
    print("\nprocessed image ",new_path+'/'+each_folder+'/'+each_file.replace("json","png"))

In [60]:
process_single_file

<function __main__.process_single_file(args)>

In [61]:
def preprocess_annotations(split):
    
    combined_path = path + 'gtFine/'+ split + '/'
    new_path = path+'label_processed'+'/'+ split
    print("combined_path: ", combined_path)
    print("new_path: ", new_path)
    
    folders = os.listdir(combined_path)
    print("Folders: ", folders)
    
    try:
        os.mkdir(new_path)
    except Exception as e:
        print(e)
        print(new_path, " already exist")

    for each_folder in folders:
        
        combined_folder_path = combined_path+'/'+each_folder

        files = os.listdir(combined_folder_path)
        files = files[0:1]
        print("File length is: ", len(files))
        
        try:
            os.mkdir(new_path+'/'+each_folder)
        except:
            print(new_path+'/'+each_folder," already exist")

        
        pool_args = []
        
        for each_file in files:
            
            pool_args.append([each_file,each_folder,combined_folder_path,new_path])
            
        pool = Pool(processes=20)
        pool.map(process_single_file,pool_args)
        pool.close()
        pool.join
            

In [62]:
for each in data_split:
    print("each: ", each)
    preprocess_annotations(each)

each:  train
combined_path:  /Users/rajkrishnanv/study/CSC2516/CSC2516_Final_Project/data_subset/gtFine/train/
new_path:  /Users/rajkrishnanv/study/CSC2516/CSC2516_Final_Project/data_subset/label_processed/train
Folders:  ['.DS_Store', '3']
[Errno 2] No such file or directory: '/Users/rajkrishnanv/study/CSC2516/CSC2516_Final_Project/data_subset/label_processed/train'
/Users/rajkrishnanv/study/CSC2516/CSC2516_Final_Project/data_subset/label_processed/train  already exist


NotADirectoryError: [Errno 20] Not a directory: '/Users/rajkrishnanv/study/CSC2516/CSC2516_Final_Project/data_subset/gtFine/train//.DS_Store'

In [29]:
def preprocess_annotations(split):
    
    combined_path = path+'gtFine/'+split+'/'
    new_path = path+'label_processed'+'/'+split
    
    folders = os.listdir(combined_path)
    print("Folders: ", folders)
    
    try:
        os.mkdir(new_path)
    except Exception as e:
        print(e)
        print(new_path, " already exist")

    for each_folder in folders:
        
        combined_folder_path = combined_path+each_folder

        files = os.listdir(combined_folder_path)
        files = files[0:1] # process only the first file
        print("File length is: ", len(files))
        
        try:
            os.mkdir(new_path+'/'+each_folder)
        except:
            print(new_path+'/'+each_folder," already exist")

        pool_args = []
        
        for each_file in files:
            inp = [each_file,each_folder,combined_folder_path,new_path]
            print(inp)
            pool_args.append(inp)
        
        with Pool(processes=1) as pool:
            # apply the process_single_file function to each set of arguments in pool_args
            pool.map(process_single_file,pool_args)